In [173]:
%reset -f
import pandas as pd
import requests
import json

In [174]:
# api endpoints
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
player_history_url = 'https://fantasy.premierleague.com/api/element-summary/{player_id}/'

In [175]:
# making request
response = requests.get(url)

# check status
if response.status_code == 200:
    print('The API request was successful.')
else:
    print(f'Error: The API request failed with status code {response.status_code}.')

The API request was successful.


In [176]:
# pandas settings
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100

In [177]:
# get all players
response = requests.get(url)
players = response.json()['elements']

#print(json.dumps(players, indent=4, sort_keys=True))

In [178]:
# mapping positions
element_types = response.json()['element_types']
position_mapping = {}

for pos in element_types:
    position_mapping[pos['id']] = pos['singular_name']

In [179]:
# extract player info
player_info = []

for player in players:
    player_id = player['id']
    name = player['first_name'] + ' ' + player['second_name']
    position = position_mapping[player['element_type']]
    points = player['total_points']
    bonus = player['bonus']
    value = player['now_cost']/10
    minutes = player['minutes']
    ppg = player['points_per_game']  
    goals = player['goals_scored']
    xg = player['expected_goals']
    xg90 = player['expected_goals_per_90']
    assists = player['assists']
    xa = player['expected_assists']
    xa90 = player['expected_assists_per_90']
    cs = player['clean_sheets']
    goals_conc = player['goals_conceded']
    xgc = player['expected_goals_conceded']
    xgc90 = player['expected_goals_conceded_per_90']
    saves = player['saves']
    influence = player['influence']
    threat = player['threat']  
    creativity = player['creativity']
    transfers_in = player['transfers_in_event']
    transfers_out = player['transfers_out_event']
  
    player_info.append({
        'name': name,
        'pos': position[:1],
        'val': value,
        'mins': minutes,
        'pts': points,
        'pts-bns': int(points) - bonus,
        'ppg': ppg,
        'g': goals,
        'xg': float(xg),
        'xg90': xg90,
        'ass': assists,
        'xa': float(xa),
        'xa90': xa90,
        'cs': cs,
        'gc': goals_conc,
        'xgc': float(xgc),
        'xgc90': xgc90,
        'saves': saves,
        'influence': influence,
        'threat': threat,
        'creativity': creativity,
        'tra_in': transfers_in,
        'tra_out': transfers_out
    })
    
df = pd.DataFrame(player_info)

In [180]:
# midfielders
mid = df.drop(columns=['cs', 'gc','saves','xgc','xgc90'])
mid.insert(6, 'xp', mid.assign(xp = mid['xa'] * 3 + mid['xg'] * 5)['xp'])
filtered_mid = mid[(mid['pos'] == 'M') & ((mid['xg90'] > 0.2) | (mid['xa90'] > 0.2)) & (mid['mins'] > 180) ]
sorted_mid = filtered_mid.sort_values(by='xp', ascending=False)

sorted_mid

,name,pos,val,mins,pts,pts-bns,xp,ppg,g,xg,xg90,ass,xa,xa90,influence,threat,creativity,tra_in,tra_out
118,Bryan Mbeumo,M,6.8,360,33,29,22.86,8.2,4,4.02,1.00,0,0.92,0.23,164.0,159.0,108.6,96767,35291
17,Bukayo Saka,M,8.7,358,25,21,15.60,6.2,2,2.16,0.54,1,1.60,0.40,150.4,166.0,207.9,32157,41934
453,Bruno Borges Fernandes,M,8.5,360,18,15,15.28,4.5,1,2.00,0.50,1,1.76,0.44,129.0,85.0,188.6,22412,51009
368,Mohamed Salah,M,12.5,341,23,23,14.28,5.8,2,2.49,0.66,2,0.61,0.16,131.8,222.0,101.2,23361,42943
620,James Maddison,M,7.8,337,29,25,13.40,7.2,2,1.60,0.43,2,1.80,0.48,162.2,171.0,168.4,227088,11672
646,Jarrod Bowen,M,7.1,360,30,25,13.40,7.5,3,2.38,0.60,1,0.50,0.12,147.0,177.0,61.6,70850,31317
264,Eberechi Eze,M,6.3,360,15,14,11.49,3.8,1,1.74,0.44,0,0.93,0.23,101.6,128.0,192.7,8018,35800
226,Enzo Fernández,M,5.0,360,5,5,10.64,1.2,0,1.36,0.34,0,1.28,0.32,65.6,61.0,193.3,6558,20364
291,Abdoulaye Doucouré,M,5.5,360,14,12,10.43,3.5,1,1.93,0.48,0,0.26,0.06,75.6,165.0,86.1,987,722
690,Pedro Lomba Neto,M,5.5,360,20,17,9.66,5.0,0,1.11,0.28,3,1.37,0.34,109.4,106.0,202.3,2391,792


In [181]:
# forwards
forw = df.drop(columns=['cs', 'gc','saves','xgc','xgc90'])
forw.insert(6, 'xp', forw.assign(xp = forw['xa'] * 3 + forw['xg'] * 4)['xp'])
filtered_forw = forw[(forw['pos'] == 'F') & ((forw['xg90'] > 0.2) | (forw['xa90'] > 0.2)) & (forw['mins'] > 180) ]
sorted_forw = filtered_forw.sort_values(by='xp', ascending=False)

sorted_forw

,name,pos,val,mins,pts,pts-bns,xp,ppg,g,xg,xg90,ass,xa,xa90,influence,threat,creativity,tra_in,tra_out
433,Erling Haaland,F,14.1,349,39,33,18.70,9.8,6,4.54,1.17,1,0.18,0.05,224.4,248.0,71.8,42108,3371
235,Nicolas Jackson,F,7.2,352,10,9,13.72,2.5,1,2.95,0.75,0,0.64,0.16,69.2,187.0,61.8,12793,163492
263,Odsonne Edouard,F,5.5,339,24,20,10.68,6.0,3,2.49,0.66,0,0.24,0.06,120.6,241.0,39.2,16921,7127
144,Evan Ferguson,F,5.9,205,25,22,10.57,6.2,4,2.56,1.12,0,0.11,0.05,147.8,163.0,18.7,206322,12073
128,Yoane Wissa,F,6.2,329,24,19,10.08,6.0,2,2.46,0.67,1,0.08,0.02,84.2,152.0,27.0,24271,111069
422,Julián Álvarez,F,6.7,359,28,24,9.53,7.0,2,1.94,0.49,3,0.59,0.15,115.8,137.0,170.2,236568,12718
698,Matheus Santos Carneiro Da Cunha,F,5.5,340,12,12,8.82,3.0,1,1.92,0.51,0,0.38,0.10,66.6,103.0,57.5,1568,2957
534,Taiwo Awoniyi,F,6.6,246,25,22,8.19,6.2,3,1.83,0.67,1,0.29,0.11,122.4,100.0,48.3,74891,11610
89,Dominic Solanke,F,6.5,360,21,19,7.67,5.2,2,1.76,0.44,1,0.21,0.05,103.2,142.0,67.6,5219,6147
147,João Pedro Junqueira de Jesus,F,5.5,226,10,9,7.40,2.5,1,1.73,0.69,0,0.16,0.06,60.0,114.0,59.6,7328,115670


In [182]:
# defenders
dfn = df.drop(columns=['saves'])
dfn.insert(6, 'xp', dfn.assign(xp = dfn['xa'] * 3 + dfn['xg'] * 6 + dfn['cs'] * 4 )['xp'])
filtered_dfn = dfn[(dfn['pos'] == 'D') & (dfn['mins'] > 180)]
sorted_dfn = filtered_dfn.sort_values(by='xp', ascending=False)

sorted_dfn

,name,pos,val,mins,pts,pts-bns,xp,ppg,g,xg,xg90,ass,xa,xa90,cs,gc,xgc,xgc90,influence,threat,creativity,tra_in,tra_out
34,Matty Cash,D,4.6,356,21,18,17.77,5.2,2,2.15,0.54,0,0.29,0.07,1,8,6.68,1.69,117.4,140.0,22.5,129097,19759
222,Ben Chilwell,D,5.8,295,15,13,11.98,3.8,0,0.70,0.21,1,1.26,0.38,1,4,3.16,0.96,47.2,69.0,127.0,38395,68429
626,Cristian Romero,D,4.7,283,27,26,11.81,6.8,2,0.62,0.20,0,0.03,0.01,2,2,4.01,1.28,149.4,46.0,3.2,194615,6308
622,Pedro Porro,D,4.9,267,16,16,11.39,5.3,0,0.37,0.12,1,0.39,0.13,2,2,3.75,1.26,61.4,23.0,46.5,32860,8079
246,Axel Disasi,D,5.1,360,19,16,10.39,4.8,1,0.67,0.17,0,0.79,0.20,1,5,4.22,1.06,93.4,64.0,19.8,20173,23023
634,Destiny Udogie,D,4.7,330,23,19,10.37,5.8,0,0.15,0.04,2,0.49,0.13,2,4,5.55,1.51,87.6,49.0,55.3,137415,7862
445,Kyle Walker,D,5.2,360,16,16,9.89,4.0,0,0.04,0.01,0,0.55,0.14,2,2,2.77,0.69,37.6,21.0,82.6,39010,22182
331,Tim Ream,D,4.5,243,12,10,9.82,4.0,1,0.96,0.36,0,0.02,0.01,1,8,11.45,4.24,59.8,37.0,2.6,8596,734
536,Willy Boly,D,4.5,338,18,15,9.16,4.5,1,0.84,0.22,0,0.04,0.01,1,6,6.11,1.63,141.0,108.0,11.8,9458,670
420,Manuel Akanji,D,5.1,270,15,14,8.84,5.0,0,0.03,0.01,0,0.22,0.07,2,1,2.04,0.68,53.8,5.0,23.8,7976,32192


In [183]:
# goalies
gk = df.drop(columns=['xg','g','xg90','ass','xa','xa90','influence','threat','creativity'])
gk.insert(6, 'xp', gk.assign(xp = gk['cs'] * 4 + gk['xgc'] * -0.5 + gk['saves'] * 0.33 + gk['mins'] / 45)['xp'])
filtered_gk = gk[(gk['pos'] == 'G') & (gk['mins'] > 180)]
sorted_gk = filtered_gk.sort_values(by='xp', ascending=False)

sorted_gk

,name,pos,val,mins,pts,pts-bns,xp,ppg,cs,gc,xgc,xgc90,saves,tra_in,tra_out
635,Guglielmo Vicario,G,5.1,360,21,18,18.265,5.2,2,4,6.03,1.51,16,45835,12922
430,Ederson Santana de Moraes,G,5.6,360,17,17,16.265,4.2,2,2,2.77,0.69,5,37822,27055
351,Alisson Ramses Becker,G,5.5,360,18,16,15.325,4.5,1,3,5.23,1.31,18,30925,7416
692,José Malheiro de Sá,G,5.0,360,17,14,15.070,4.2,1,8,7.06,1.76,20,3613,2769
527,Matt Turner,G,4.0,360,12,12,14.090,3.0,1,6,6.38,1.60,16,27053,10420
111,Mark Flekken,G,4.5,360,14,13,14.020,3.5,1,5,4.54,1.14,13,8753,4977
325,Bernd Leno,G,4.5,360,19,14,13.240,4.8,1,10,12.04,3.01,22,52318,14330
480,André Onana,G,5.0,360,11,10,13.160,2.8,1,7,7.58,1.90,15,12885,66475
212,Robert Sánchez,G,4.5,360,12,12,11.870,3.0,1,5,4.22,1.06,6,13869,10361
15,Aaron Ramsdale,G,5.0,360,11,11,11.785,2.8,1,4,3.73,0.93,5,15740,19762
